In [1]:
import torch.nn as nn
import torch
import torch.nn.utils.rnn as rnn
import statistics
import nltk
import random
import collections
import time


In [2]:
 # Set cuda if available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [3]:
## Dictionary class 선언
class Dictionary(object):
    def __init__(self, dataset, size):
        ## init vocab ##
        self.word2idx = {'<pad>':0, '<sos>': 1, '<eos>': 2, '<unk>': 3} # 사전 (unk-신조어, 욕)
        self.idx2word = ['<pad>', '<sos>', '<eos>', '<unk>']

        self.build_dict(dataset, size)

    def __call__(self, word):
        return self.word2idx.get(word, self.word2idx['<unk>']) # if word does not exist in vocab then return unk idx

    def add_word(self, word):
        if word not in self.word2idx:
            self.idx2word.append(word)
            self.word2idx[word] = len(self.idx2word) - 1
        return self.word2idx[word]

    def build_dict(self, dataset, dict_size):
        """Tokenize a text file."""
        total_words = (word for sent in dataset for word in sent) # store all words into tuple
        word_freq = collections.Counter(total_words) # count the number of each word: ex) ('The': 10000, 'a': 5555, ...)
        vocab = sorted(word_freq.keys(), key=lambda word: (-word_freq[word], word)) # sort by frequency
        vocab = vocab[:dict_size]
        for word in vocab:
            self.add_word(word)

    def __len__(self):
        return len(self.idx2word)


In [4]:
## Brown dataset Preprocessing (NLTK)
def brown_dataset(min=5, max=30):
    nltk.download('brown')

    # get sentences with the length between min and max
    # convert all words into lower-case
    all_seq = [[token.lower() for token in seq] for seq in nltk.corpus.brown.sents() 
               if min <= len(seq) <= max]

    random.shuffle(all_seq) # shuffle
    return all_seq

In [5]:
## Download Brown dataset
dataset = brown_dataset()

## print some part
print(dataset[0])
print(dataset[1])
print(dataset[2])

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\cptas\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


['the', 'designation', 'of', 'five', 'catholic', 'theologians', 'to', 'attend', 'the', 'world', 'council', 'of', 'churches', 'assembly', 'in', 'new', 'delhi', 'as', '``', 'official', "''", 'observers', 'reverses', 'the', "church's", 'earlier', 'stand', '.']
['a', 'growl', ',', 'a', 'nod', ',', 'was', 'usually', 'enough', '.']
['bishop', 'rushed', 'on', 'deck', 'to', 'grab', 'a', '20-mm', 'gun', ',', 'pumping', 'out', '400', 'rounds', 'before', 'sticks', 'of', 'three', 'bombs', 'each', 'crashed', 'into', 'holds', 'one', ',', 'three', 'and', 'five', '.']


In [6]:
## Data handler class 선언

class Corpus(object):
    def __init__(self, dataset, device, dict_size=20000, train_ratio=0.95):
        train_size = int(len(dataset) * train_ratio)
        valid_size = len(dataset) - train_size
        self.device = device
        self.dictionary = Dictionary(dataset, dict_size)
        self.train = dataset[:train_size] # 0 ~ train_size
        self.valid = dataset[train_size:train_size + valid_size] # train_size: train_size+valid_size

    def indexing(self, dat):
        src_idxes = [] # 모델 입력
        tgt_idxes = [] # 모델 정답
        for sent in dat:
            src_idx = [self.dictionary('<sos>')] + [self.dictionary(word) for word in sent]
            tgt_idx = [self.dictionary(word) for word in sent] + [self.dictionary('<eos>')]
            src_idxes.append(torch.tensor(src_idx).type(torch.int64))
            tgt_idxes.append(torch.tensor(tgt_idx).type(torch.int64))

        src_idxes = rnn.pad_sequence(src_idxes, batch_first=True).to(self.device) # shape = [B, L]
        tgt_idxes = rnn.pad_sequence(tgt_idxes, batch_first=True).to(self.device).view(-1) # flatten shape = [B * L]

        return src_idxes, tgt_idxes

    def batch_iter(self, batch_size, isTrain=True):
        dat = self.train if isTrain else self.valid
        if isTrain:
            random.shuffle(dat)

        for i in range(len(dat) // batch_size):
            batch = dat[i * batch_size: (i+1) * batch_size]
            src, tgt = self.indexing(batch)
            yield {'src': src, 'tgt': tgt}

In [7]:
corpus = Corpus(dataset, device)

In [8]:
# Dictionary 확인

for i, (key, val) in enumerate(corpus.dictionary.word2idx.items()):
    print('word:  {:10s} | index: {:5d} '.format(key, val))
    if i == 20:
        break


word:  <pad>      | index:     0 
word:  <sos>      | index:     1 
word:  <eos>      | index:     2 
word:  <unk>      | index:     3 
word:  the        | index:     4 
word:  .          | index:     5 
word:  ,          | index:     6 
word:  of         | index:     7 
word:  and        | index:     8 
word:  to         | index:     9 
word:  a          | index:    10 
word:  in         | index:    11 
word:  was        | index:    12 
word:  he         | index:    13 
word:  is         | index:    14 
word:  ''         | index:    15 
word:  ``         | index:    16 
word:  it         | index:    17 
word:  that       | index:    18 
word:  for        | index:    19 
word:  ;          | index:    20 


In [9]:
## indexing 함수 결과 확인

# case : 단일 문장 입력 시. 
sent = [dataset[1]]
idx_src, idx_tgt = corpus.indexing(sent)

print(sent)
print(idx_src) # <SOS> index로 시작
print(idx_tgt) # <EOS> index로 종료

print('-' * 90)
## case : 복수 문장 입력 시 (batching)
batch = [dataset[0], dataset[1]]
idx_src, idx_tgt = corpus.indexing(batch)

print(batch)
print(idx_src) # 가장 길이가 긴 문장 (dataset[0]) 보다 짧은 문장 (dataset[1]) 의 경우 남는 길이만큼 padding=0 삽입 확인.
print(idx_tgt)

[['a', 'growl', ',', 'a', 'nod', ',', 'was', 'usually', 'enough', '.']]
tensor([[    1,    10, 13500,     6,    10,  6268,     6,    12,   438,   210,
             5]])
tensor([   10, 13500,     6,    10,  6268,     6,    12,   438,   210,     5,
            2])
------------------------------------------------------------------------------------------
[['the', 'designation', 'of', 'five', 'catholic', 'theologians', 'to', 'attend', 'the', 'world', 'council', 'of', 'churches', 'assembly', 'in', 'new', 'delhi', 'as', '``', 'official', "''", 'observers', 'reverses', 'the', "church's", 'earlier', 'stand', '.'], ['a', 'growl', ',', 'a', 'nod', ',', 'was', 'usually', 'enough', '.']]
tensor([[    1,     4, 13042,     7,   319,  1427,  8848,     9,  1858,     4,
           148,  1256,     7,  1429,  2745,    11,    84, 16806,    25,    16,
          1318,    15,  5012, 19729,     4,  9200,   819,   718,     5],
        [    1,    10, 13500,     6,    10,  6268,     6,    12,   438,   210,
     

In [10]:
## RNN Language model 선언

# Define network
class RNNModel(nn.Module):
    def __init__(self, ntoken, hidden_size, nlayers, dropout=0.1): #정의부분
        super(RNNModel, self).__init__()
        self.drop = nn.Dropout(dropout)
        self.embeddings = nn.Embedding(ntoken, hidden_size, padding_idx=0)
        self.rnn = nn.LSTM(hidden_size, hidden_size, nlayers, dropout=dropout, 
                           batch_first=True) # [B, L, Hidden_DIM], nlayers->쌓는거
        self.output_layer = nn.Linear(hidden_size, ntoken)
        self.sm = nn.LogSoftmax(dim=1)

        self.ntoken = ntoken
        self.hidden_size = hidden_size
        self.nlayers = nlayers

        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.embeddings.weight.data.uniform_(-initrange, initrange)
        self.output_layer.weight.data.uniform_(-initrange, initrange)
        self.output_layer.bias.data.zero_()

    def forward(self, input, hidden): #정의된 것을 바탕으로 어떻게 동작하는지
        emb = self.embeddings(input) # emb = (batch, length, dim) input shape: [batch size, length]
        output, hidden = self.rnn(emb, hidden) # output = (batch. length. dim)
        output = self.drop(output)
        output = self.output_layer(output) # output = (batch, length, vocab_size)
        output = output.view(-1, self.ntoken) # output = (batch * length, vocab_size)

        return self.sm(output), hidden #log prob

    def init_hidden(self, bsz):
        weight = next(self.parameters()) # to set init tensor with the same torch.dtype and torch.device
        return (weight.new_zeros(self.nlayers, bsz, self.hidden_size),
                weight.new_zeros(self.nlayers, bsz, self.hidden_size))


In [15]:
# Hyperparameters
batch_size = 40
hidden_size = 100
dropout = 0.2
max_epoch = 10

# build model
ntokens = len(corpus.dictionary)
model = RNNModel(ntokens, hidden_size, 2, dropout).to(device)
isTrain=True # Flag variable

# set loss func and optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = nn.NLLLoss(ignore_index=0, reduction='mean')

In [ ]:
##### Training / Evaluation Parts #######

In [12]:
# accuracy
def cal_acc(scores, target):
    pred = scores.max(-1)[1]
    non_pad = target.ne(0)
    num_correct = pred.eq(target).masked_select(non_pad).sum().item() 
    num_non_pad = non_pad.sum().item()
    return 100 * (num_correct / num_non_pad)

In [13]:
# train func.
def train():
    model.train() # Turn on training mode which enables dropout.
    mean_loss = []
    mean_acc = []
    start_time = time.time()

    for batch in corpus.batch_iter(batch_size):
        hidden = model.init_hidden(batch_size) # zero vectors for init hidden
        target = batch['tgt'] # flattened target 
        optimizer.zero_grad()
        output, hidden = model(batch['src'], hidden) # output = flatten output = [Batch_size * Length, vocab_size]
                            # = model.forward(batch)
        # output shape = (batch * length, vocab_size)
        # target shape = (batch * length)   --> (batch * length, vocab_size) 로 one-hot distribtuion으로 내부적으로 변환되어 비교 수행
        loss = criterion(output, target) # compare between vocab_prob and answer_prob(one-hot converted)
        loss.backward()
        optimizer.step()

        mean_loss.append(loss.item())
        mean_acc.append(cal_acc(output, target))

    total_time = time.time() - start_time
    mean_acc = statistics.mean(mean_acc)
    mean_loss = statistics.mean(mean_loss)

    return mean_loss, total_time, mean_acc

In [14]:
# evaluation func.
def evaluate():
    model.eval() # Turn off dropout
    mean_loss = []
    mean_acc = []

    for batch in corpus.batch_iter(batch_size, isTrain=False):
        with torch.no_grad():
            hidden = model.init_hidden(batch_size)
            target = batch['tgt']
            output, hidden = model(batch['src'], hidden)
            loss = criterion(output, target)
            mean_loss.append(loss.item())
            mean_acc.append(cal_acc(output, target))

    mean_acc = statistics.mean(mean_acc)
    mean_loss = statistics.mean(mean_loss)

    return mean_loss, mean_acc

In [ ]:
if isTrain: # set False if you don't need to train model
    start_time = time.time()

    for epoch in range(1, max_epoch+1):
        loss, epoch_time, accuracy = train()
        print('epoch {:4d} | times {:3.3f} |  loss: {:3.3f} | accuracy: {:3.2f}'.format(epoch+1, epoch_time, loss, accuracy))

        if epoch % 10 == 0:
            loss, accuracy = evaluate()
            print('=' * 60)
            print('Evaluation | loss: {:3.3f} | accuracy: {:3.2f}'.format(epoch+1, epoch_time, loss, accuracy))
            print('=' * 60)

    with open('model.pt', 'wb') as f:
        print('save model at: ./model.pt')
        torch.save(model, f)

epoch    2 | times 3586.584 |  loss: 6.199 | accuracy: 15.79
epoch    3 | times 2908.004 |  loss: 5.581 | accuracy: 20.48
epoch    4 | times 3138.720 |  loss: 5.359 | accuracy: 21.87
epoch    5 | times 3731.240 |  loss: 5.192 | accuracy: 22.68


In [ ]:
def pred_sent_prob(sent):
    import numpy as np
    model.eval()
    with torch.no_grad():
        inputs = corpus.indexing(sent)
        # 1. construct input for the model  : index 활용
        # do someting about #1....
        
        
        hidden = model.init_hidden(inputs)
        output, hidden = model(batch['src'], hidden)
        # 2. set init hidden : 첫번째 step에 대한 zero vector
        # do someting about #2....
        
        
        tag_scores = model.forward(output)
        # 3. get model output : vocab 크기의 log softmax 결과값 출력
        # do someting about #3....
        
        
        log_prob = criterion(output, target)
        # 4. get word log_prob : 찾고자 하는 단어의 log 확률값 추출
        # do someting about #4....
        
        
        sent_prob = cal_acc(log_prob, target)
        # 5. comput sentence log_prob by summing each of word log_prob : 4의 각 결과를 합
        # do someting about #5....
        
        return sent_prob

In [24]:
# load saved model
with open('./model.pt', 'rb') as f:
    print('load model from: ./model.pt')
    model = torch.load(f).to(device)

    print('log prob of [the dog bark .]: {:3.3f}'.format(pred_sent_prob([['the', 'dog', 'bark', '.']])))
    print('log prob of [the cat bark .]: {:3.3f}'.format(pred_sent_prob([['the', 'cat', 'bark', '.']])))

    print('log prob of [boy am a i .]: {:3.3f}'.format(pred_sent_prob([['boy', 'am', 'a', 'i', '.']])))
    print('log prob of [i am a boy .]: {:3.3f}'.format(pred_sent_prob([['i', 'am', 'a', 'boy', '.']])))


load model from: ./model.pt
log prob of [the dog bark .]: -46.537
log prob of [the cat bark .]: -51.715
log prob of [boy am a i .]: -42.927
log prob of [i am a boy .]: -18.946
